# Phase 1 Ablation: PPO vs GNS vs PhysRobot-SV

**Protocol** (from EXPERIMENT_DESIGN.md Phase 1 / SPEC.md):
- 3 methods x 5 seeds x 500K timesteps
- Seeds: [42, 123, 256, 789, 1024]
- Eval: 100 episodes per seed (deterministic), seeds 10000-10099
- OOD: box mass sweep [0.1, 0.25, 0.5, 0.75, 1.0, 2.0, 5.0]
- Stats: mean +/- std, Welch t-test, 95% CI

PhysRobot-SV uses the full Scalarization-Vectorization pipeline from
`sv_message_passing.py` (undirected-pair Newton 3rd law, stop-grad fusion).


In [ ]:
%%time
!pip install mujoco gymnasium stable-baselines3[extra] torch torch-geometric matplotlib pandas scipy -q
import torch
print(f'PyTorch {torch.__version__}, CUDA {torch.cuda.is_available()}')
if torch.cuda.is_available():
    print(f'  GPU: {torch.cuda.get_device_name(0)}')


In [ ]:
from google.colab import drive
import os, json, time
drive.mount('/content/drive')
SAVE_DIR = '/content/drive/MyDrive/physrobot_phase1_ablation'
for d in ['', 'models', 'results', 'logs', 'figures']:
    os.makedirs(f'{SAVE_DIR}/{d}' if d else SAVE_DIR, exist_ok=True)
print(f'Save dir: {SAVE_DIR}')


In [ ]:
# ============================================================
# PushBoxEnv  (16-dim obs, 2-dim action)
# Canonical source: SPEC.md section 1
# ============================================================
import numpy as np
import mujoco
import gymnasium as gym
from gymnasium import spaces

MUJOCO_XML = r'''<mujoco model="push_box">
  <compiler angle="degree" coordinate="local" inertiafromgeom="true"/>
  <option timestep="0.002" integrator="Euler" gravity="0 0 -9.81">
    <flag warmstart="enable"/>
  </option>
  <asset>
    <texture builtin="checker" height="100" name="texplane" rgb1="0.2 0.2 0.2" rgb2="0.3 0.3 0.3" type="2d" width="100"/>
    <material name="MatPlane" reflectance="0.3" shininess="0.5" specular="0.5" texrepeat="3 3" texture="texplane"/>
  </asset>
  <default>
    <joint armature="0.01" damping="0.1" limited="true"/>
    <geom conaffinity="1" condim="3" contype="1" friction="0.5 0.005 0.0001" margin="0.001" rgba="0.8 0.6 0.4 1"/>
  </default>
  <worldbody>
    <light directional="true" diffuse="0.8 0.8 0.8" pos="0 0 3" dir="0 0 -1"/>
    <geom name="floor" type="plane" size="3 3 0.1" rgba="0.8 0.8 0.8 1" material="MatPlane"/>
    <body name="arm_base" pos="0 0 0.02">
      <geom name="base_geom" type="cylinder" size="0.05 0.02" rgba="0.3 0.3 0.3 1"/>
      <body name="upper_arm" pos="0 0 0.02">
        <joint name="shoulder" type="hinge" axis="0 0 1" range="-180 180" damping="0.5"/>
        <geom name="upper_arm_geom" type="capsule" fromto="0 0 0 0.4 0 0" size="0.025" rgba="0.5 0.5 0.8 1"/>
        <body name="forearm" pos="0.4 0 0">
          <joint name="elbow" type="hinge" axis="0 0 1" range="-180 180" damping="0.5"/>
          <geom name="forearm_geom" type="capsule" fromto="0 0 0 0.3 0 0" size="0.025" rgba="0.5 0.5 0.8 1"/>
          <site name="endeffector" pos="0.3 0 0" size="0.03" rgba="1 0.5 0 0.8"/>
        </body>
      </body>
    </body>
    <body name="box" pos="0.35 0 0.05">
      <freejoint name="box_freejoint"/>
      <geom name="box_geom" type="box" size="0.05 0.05 0.05" mass="0.5"
            rgba="0.2 0.8 0.2 1" friction="0.5 0.005 0.0001"/>
    </body>
    <site name="goal" pos="0.5 0.3 0.02" size="0.06" rgba="1 0 0 0.4" type="sphere"/>
  </worldbody>
  <actuator>
    <motor name="shoulder_motor" joint="shoulder" gear="1" ctrllimited="true" ctrlrange="-10 10"/>
    <motor name="elbow_motor"    joint="elbow"    gear="1" ctrllimited="true" ctrlrange="-10 10"/>
  </actuator>
</mujoco>'''


class PushBoxEnv(gym.Env):
    """PushBox V2 (SPEC-aligned): 16-dim obs, 2-dim action, progress reward."""

    def __init__(self, box_mass=0.5):
        super().__init__()
        self.model = mujoco.MjModel.from_xml_string(MUJOCO_XML)
        self.data = mujoco.MjData(self.model)
        self.box_mass = box_mass
        self._set_box_mass(box_mass)
        self._ee_sid = mujoco.mj_name2id(
            self.model, mujoco.mjtObj.mjOBJ_SITE, 'endeffector')
        self.action_space = spaces.Box(-10.0, 10.0, (2,), np.float32)
        self.observation_space = spaces.Box(-np.inf, np.inf, (16,), np.float32)
        self.goal_pos = np.array([0.5, 0.3, 0.02])
        self.max_steps = 500
        self.success_threshold = 0.15
        self._step = 0
        self._prev_d_bg = None
        self._prev_d_eb = None

    # ---- helpers ----
    def _set_box_mass(self, m):
        bid = mujoco.mj_name2id(self.model, mujoco.mjtObj.mjOBJ_BODY, 'box')
        self.model.body_mass[bid] = m

    def set_box_mass(self, m):
        self.box_mass = m
        self._set_box_mass(m)

    def _obs(self):
        jp = self.data.qpos[:2].copy()
        jv = self.data.qvel[:2].copy()
        ee = self.data.site_xpos[self._ee_sid].copy()
        bp = self.data.qpos[2:5].copy()
        bv = self.data.qvel[2:5].copy()
        gp = self.goal_pos.copy()
        return np.concatenate([jp, jv, ee, bp, bv, gp]).astype(np.float32)

    # ---- gym API ----
    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        mujoco.mj_resetData(self.model, self.data)
        if seed is not None:
            np.random.seed(seed)
        self.data.qpos[0] = np.random.uniform(-0.5, 0.5)
        self.data.qpos[1] = np.random.uniform(-0.5, 0.5)
        self.data.qpos[2] = np.random.uniform(0.25, 0.45)
        self.data.qpos[3] = np.random.uniform(-0.15, 0.15)
        self.data.qpos[4] = 0.05
        self.data.qpos[5:9] = [1, 0, 0, 0]
        self.data.qvel[:] = 0.0
        mujoco.mj_forward(self.model, self.data)
        self._step = 0
        ee = self.data.site_xpos[self._ee_sid]
        bp = self.data.qpos[2:5]
        self._prev_d_bg = np.linalg.norm(bp[:2] - self.goal_pos[:2])
        self._prev_d_eb = np.linalg.norm(ee[:2] - bp[:2])
        return self._obs(), self._info()

    def _info(self):
        bp = self.data.qpos[2:5]
        d = np.linalg.norm(bp[:2] - self.goal_pos[:2])
        return {'distance_to_goal': d, 'success': d < self.success_threshold,
                'box_mass': self.box_mass}

    def step(self, action):
        self.data.ctrl[:] = action
        for _ in range(5):
            mujoco.mj_step(self.model, self.data)
        ee = self.data.site_xpos[self._ee_sid].copy()
        bp = self.data.qpos[2:5].copy()
        d_eb = np.linalg.norm(ee[:2] - bp[:2])
        d_bg = np.linalg.norm(bp[:2] - self.goal_pos[:2])
        reach_prog = self._prev_d_eb - d_eb
        push_prog  = self._prev_d_bg - d_bg
        self._prev_d_eb = d_eb
        self._prev_d_bg = d_bg
        reward = (
            0.5 * (-d_eb) + 1.0 * (-d_bg)
            + 10.0 * reach_prog + 20.0 * push_prog
            - 0.01 * np.sum(action ** 2)
        )
        success = d_bg < self.success_threshold
        if success:
            reward += 500.0 + (self.max_steps - self._step) * 1.0
        self._step += 1
        return self._obs(), reward, success, self._step >= self.max_steps, self._info()

    def render(self): pass
    def close(self): pass


def make_env(box_mass=0.5):
    def _f():
        return PushBoxEnv(box_mass=box_mass)
    return _f


# Quick check
_e = PushBoxEnv(); _o, _ = _e.reset()
print(f'obs {_o.shape}  action {_e.action_space.shape}  threshold {_e.success_threshold}m')
_e.close()
print('Environment OK')


In [ ]:
# ============================================================
# Agent definitions: PPO / GNS / PhysRobot-SV
# ============================================================
import torch
import torch.nn as nn
from typing import Tuple
from stable_baselines3 import PPO
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.callbacks import BaseCallback

try:
    from torch_geometric.nn import MessagePassing as PyGMP
    from torch_geometric.data import Data as PyGData, Batch as PyGBatch
    HAS_PYG = True
except ImportError:
    HAS_PYG = False
    print('WARNING: torch_geometric unavailable; GNS uses MLP fallback')


# ---------- shared ----------
SEEDS = [42, 123, 256, 789, 1024]
EVAL_SEEDS = list(range(10000, 10100))
TOTAL_TIMESTEPS = 500_000
N_ENVS = 4
BOX_MASS_TRAIN = 0.5
OOD_MASSES = [0.1, 0.25, 0.5, 0.75, 1.0, 2.0, 5.0]

PPO_KWARGS = dict(
    learning_rate=3e-4, n_steps=2048, batch_size=64,
    n_epochs=10, gamma=0.99, gae_lambda=0.95,
    clip_range=0.2, ent_coef=0.01, vf_coef=0.5,
    max_grad_norm=0.5, verbose=0,
)


class SuccessCallback(BaseCallback):
    """Track episode-level successes during training."""
    def __init__(self):
        super().__init__()
        self.ep = 0
        self.ok = 0
        self.first_ok_ep = None
        self.history = []  # (ep, cumulative_sr)

    def _on_step(self):
        for i, done in enumerate(self.locals.get('dones', [])):
            if done:
                self.ep += 1
                info = self.locals['infos'][i] if i < len(self.locals['infos']) else {}
                if info.get('success', False):
                    self.ok += 1
                    if self.first_ok_ep is None:
                        self.first_ok_ep = self.ep
                if self.ep % 200 == 0:
                    self.history.append((self.ep, self.ok / self.ep))
        return True


# ========== 1. Pure PPO ==========
def make_ppo_agent(env):
    return PPO('MlpPolicy', env,
               policy_kwargs=dict(net_arch=dict(pi=[64, 64], vf=[64, 64])),
               **PPO_KWARGS)


# ========== 2. GNS V2 (lightweight graph, ~5K FE params) ==========
if HAS_PYG:
    class _GNSLayer(PyGMP):
        def __init__(self, nd, ed, hd=32):
            super().__init__(aggr='add')
            self.edge_mlp = nn.Sequential(
                nn.Linear(2*nd+ed, hd), nn.ReLU(), nn.Linear(hd, ed))
            self.node_mlp = nn.Sequential(
                nn.Linear(nd+ed, hd), nn.ReLU(), nn.Linear(hd, nd))
        def forward(self, x, edge_index, edge_attr):
            return self.propagate(edge_index, x=x, edge_attr=edge_attr)
        def message(self, x_i, x_j, edge_attr):
            return self.edge_mlp(torch.cat([x_i, x_j, edge_attr], -1))
        def update(self, agg, x):
            return self.node_mlp(torch.cat([x, agg], -1))

    def _obs2graph(obs):
        B = obs.shape[0]; dev = obs.device; gs = []
        for i in range(B):
            o = obs[i]
            ep, bp, bv = o[4:7], o[7:10], o[10:13]
            ev = torch.zeros(3, device=dev)
            nf = torch.stack([torch.cat([ep, ev]), torch.cat([bp, bv])])
            ei = torch.tensor([[0,1],[1,0]], dtype=torch.long, device=dev).t().contiguous()
            r01 = bp - ep; r10 = ep - bp
            ea = torch.stack([torch.cat([r01, r01.norm().unsqueeze(0)]),
                              torch.cat([r10, r10.norm().unsqueeze(0)])])
            gs.append(PyGData(x=nf, edge_index=ei, edge_attr=ea))
        return PyGBatch.from_data_list(gs)

    class GNSExtractor(BaseFeaturesExtractor):
        def __init__(self, observation_space, features_dim=64):
            super().__init__(observation_space, features_dim)
            h = 32; ed = 4
            self.ne = nn.Sequential(nn.Linear(6, h), nn.ReLU())
            self.ee = nn.Sequential(nn.Linear(ed, h), nn.ReLU())
            self.gn = _GNSLayer(h, h, h)
            self.dec = nn.Linear(h, 3)
            self.proj = nn.Sequential(nn.Linear(3+16, features_dim), nn.ReLU())
        def forward(self, obs):
            g = _obs2graph(obs)
            x = self.ne(g.x); ea = self.ee(g.edge_attr)
            x = x + self.gn(x, g.edge_index, ea)
            acc = self.dec(x)
            box_acc = acc[1::2]
            return self.proj(torch.cat([box_acc, obs], -1))
else:
    class GNSExtractor(BaseFeaturesExtractor):
        """Fallback when PyG unavailable."""
        def __init__(self, observation_space, features_dim=64):
            super().__init__(observation_space, features_dim)
            self.net = nn.Sequential(
                nn.Linear(16, 64), nn.ReLU(),
                nn.Linear(64, features_dim), nn.ReLU())
        def forward(self, obs):
            return self.net(obs)


def make_gns_agent(env):
    pk = dict(
        features_extractor_class=GNSExtractor,
        features_extractor_kwargs=dict(features_dim=64),
        net_arch=dict(pi=[64, 64], vf=[64, 64]),
    )
    return PPO('MlpPolicy', env, policy_kwargs=pk, **PPO_KWARGS)


# ========== 3. PhysRobot-SV (full SV pipeline from sv_message_passing.py) ==========
EPS = 1e-7
DEG_EPS = 1e-4

def _sv_mlp(ind, hd, od):
    return nn.Sequential(nn.Linear(ind, hd), nn.LayerNorm(hd), nn.ReLU(),
                         nn.Linear(hd, od))

def build_edge_frames(pos, vel, src, dst):
    r_ij = pos[dst] - pos[src]
    d_ij = torch.norm(r_ij, dim=-1, keepdim=True)
    e1 = r_ij / (d_ij + EPS)
    v_rel = vel[dst] - vel[src]
    v_par = (v_rel * e1).sum(-1, keepdim=True) * e1
    v_perp = v_rel - v_par
    vp_n = torch.norm(v_perp, dim=-1, keepdim=True)
    nondeg = (vp_n > DEG_EPS).float()
    e2_vel = v_perp / (vp_n + EPS)
    z = torch.tensor([0.,0.,1.], device=pos.device).expand_as(e1)
    fb_raw = torch.cross(e1, z, dim=-1)
    fb_n = torch.norm(fb_raw, dim=-1, keepdim=True)
    use_y = (fb_n < DEG_EPS).float()
    y = torch.tensor([0.,1.,0.], device=pos.device).expand_as(e1)
    fb_raw = (1-use_y)*fb_raw + use_y*torch.cross(e1, y, dim=-1)
    fb_n = torch.norm(fb_raw, dim=-1, keepdim=True)
    e2_fb = fb_raw / (fb_n + EPS)
    e2 = nondeg * e2_vel + (1 - nondeg) * e2_fb
    e3 = torch.cross(e1, e2, dim=-1)
    return e1, e2, e3, r_ij, d_ij


class SVMessagePassing(nn.Module):
    """One SV round (undirected-pair, hard Newton 3rd law)."""
    def __init__(self, nd, hd=32):
        super().__init__()
        self.force_mlp = _sv_mlp(5 + 2*nd, hd, 3)
        self.node_update = _sv_mlp(nd + 3, hd, nd)

    def forward_with_forces(self, h, edge_index, pos, vel):
        N = h.size(0)
        s, d = edge_index
        mask = s < d
        pi, pj = s[mask], d[mask]
        e1, e2, e3, _, d_ij = build_edge_frames(pos, vel, pi, pj)
        vr = vel[pj] - vel[pi]
        vr_sc = (vr * e1).sum(-1, keepdim=True)
        vt_sc = (vr * e2).sum(-1, keepdim=True)
        vb_sc = (vr * e3).sum(-1, keepdim=True)
        vn_sc = torch.norm(vr, dim=-1, keepdim=True)
        sc = torch.cat([d_ij, vr_sc, vt_sc, vb_sc, vn_sc], -1)
        hs = h[pi] + h[pj]
        hd_abs = (h[pi] - h[pj]).abs()
        inp = torch.cat([sc, hs, hd_abs], -1)
        a = self.force_mlp(inp)
        F = a[:,0:1]*e1 + a[:,1:2]*e2 + a[:,2:3]*e3
        Fa = torch.zeros(N, 3, device=h.device, dtype=h.dtype)
        Fa.scatter_add_(0, pj.unsqueeze(-1).expand_as(F), F)
        Fa.scatter_add_(0, pi.unsqueeze(-1).expand_as(F), -F)
        h_new = h + self.node_update(torch.cat([h, Fa], -1))
        return h_new, Fa


class SVPhysicsCore(nn.Module):
    def __init__(self, node_in=6, hd=32, n_layers=1):
        super().__init__()
        self.encoder = _sv_mlp(node_in, hd, hd)
        self.layers = nn.ModuleList([SVMessagePassing(hd, hd) for _ in range(n_layers)])

    def forward(self, pos, vel, edge_index):
        h = self.encoder(torch.cat([pos, vel], -1))
        F = None
        for ly in self.layers:
            h, F = ly.forward_with_forces(h, edge_index, pos, vel)
        return F


class PhysRobotSVExtractor(BaseFeaturesExtractor):
    """Dual-stream: SV physics + MLP policy, stop-grad fusion."""
    def __init__(self, observation_space, features_dim=64):
        super().__init__(observation_space, features_dim)
        self.physics = SVPhysicsCore(6, 32, 1)
        self.policy_stream = nn.Sequential(
            nn.Linear(16, 64), nn.ReLU(), nn.Linear(64, features_dim), nn.ReLU())
        self.fusion = nn.Sequential(
            nn.Linear(features_dim + 3, features_dim), nn.ReLU())
        self._ei = torch.tensor([[0,1],[1,0]], dtype=torch.long).t()

    def forward(self, obs):
        B = obs.shape[0]; dev = obs.device
        z_pol = self.policy_stream(obs)
        ep = obs[:, 4:7]; bp = obs[:, 7:10]; bv = obs[:, 10:13]
        ev = torch.zeros_like(ep)
        ei = self._ei.to(dev)
        accs = []
        for i in range(B):
            p = torch.stack([ep[i], bp[i]])
            v = torch.stack([ev[i], bv[i]])
            a = self.physics(p, v, ei)
            accs.append(a[1])
        z_phys = torch.stack(accs)
        return self.fusion(torch.cat([z_pol, z_phys.detach()], -1))


def make_sv_agent(env):
    pk = dict(
        features_extractor_class=PhysRobotSVExtractor,
        features_extractor_kwargs=dict(features_dim=64),
        net_arch=dict(pi=[64, 64], vf=[64, 64]),
    )
    return PPO('MlpPolicy', env, policy_kwargs=pk, **PPO_KWARGS)


# ---- param count check ----
_tv = DummyVecEnv([make_env()])
for name, fn in [('PPO', make_ppo_agent), ('GNS', make_gns_agent), ('PhysRobot-SV', make_sv_agent)]:
    m = fn(_tv)
    n = sum(p.numel() for p in m.policy.parameters())
    print(f'{name:15s} {n:>8,} params')
    del m
_tv.close()
print('All agents OK')


In [ ]:
# ============================================================
# Training: 500K steps x 3 methods x 5 seeds
# ============================================================
import time, json, gc

METHODS = {
    'PPO':          make_ppo_agent,
    'GNS':          make_gns_agent,
    'PhysRobot-SV': make_sv_agent,
}

all_results = {}  # method -> seed -> {sr, reward, ...}
all_models  = {}  # method -> seed -> model

total_runs = len(METHODS) * len(SEEDS)
run_i = 0

for method_name, make_fn in METHODS.items():
    all_results[method_name] = {}
    all_models[method_name] = {}
    for seed in SEEDS:
        run_i += 1
        tag = f'{method_name}/seed{seed}'
        print(f'\n[{run_i}/{total_runs}] {tag}')

        torch.manual_seed(seed)
        np.random.seed(seed)

        env = DummyVecEnv([make_env(BOX_MASS_TRAIN) for _ in range(N_ENVS)])
        model = make_fn(env)
        cb = SuccessCallback()

        t0 = time.time()
        try:
            model.learn(total_timesteps=TOTAL_TIMESTEPS, callback=cb, progress_bar=False)
        except Exception as exc:
            print(f'  ERROR: {exc}')
            all_results[method_name][seed] = {'error': str(exc)}
            env.close(); del model; gc.collect(); continue
        wall = time.time() - t0

        # Evaluate in-distribution (100 ep, deterministic)
        eval_env = PushBoxEnv(box_mass=BOX_MASS_TRAIN)
        rews, succ = [], []
        for es in EVAL_SEEDS:
            obs, _ = eval_env.reset(seed=es)
            done, er = False, 0.0
            while not done:
                act, _ = model.predict(obs, deterministic=True)
                obs, r, term, trunc, info = eval_env.step(act)
                er += r; done = term or trunc
            rews.append(er)
            succ.append(int(info.get('success', False)))
        eval_env.close()

        sr = np.mean(succ)
        rec = {
            'success_rate': float(sr),
            'mean_reward': float(np.mean(rews)),
            'std_reward': float(np.std(rews)),
            'first_success_ep': cb.first_ok_ep,
            'train_time_s': wall,
            'train_episodes': cb.ep,
            'train_successes': cb.ok,
            'history': cb.history,
        }
        all_results[method_name][seed] = rec

        # Save model
        mpath = f'{SAVE_DIR}/models/{method_name}_seed{seed}'
        model.save(mpath)
        all_models[method_name][seed] = model

        print(f'  SR={sr:.0%}  reward={np.mean(rews):.0f}  '
              f'1st_ok=ep{cb.first_ok_ep}  wall={wall/60:.1f}m')

        env.close()
        gc.collect()

# Save raw results
with open(f'{SAVE_DIR}/results/phase1_raw.json', 'w') as f:
    json.dump(all_results, f, indent=2, default=str)
print(f'\nResults saved to {SAVE_DIR}/results/phase1_raw.json')

# Summary table
print('\n' + '='*65)
print(f'{"Method":15s} {"SR (mean+/-std)":18s} {"Reward":14s} {"Time/seed":10s}')
print('-'*65)
for mn in METHODS:
    srs = [all_results[mn][s]['success_rate'] for s in SEEDS
           if 'error' not in all_results[mn].get(s, {})]
    rws = [all_results[mn][s]['mean_reward'] for s in SEEDS
           if 'error' not in all_results[mn].get(s, {})]
    tms = [all_results[mn][s]['train_time_s'] for s in SEEDS
           if 'error' not in all_results[mn].get(s, {})]
    if srs:
        print(f'{mn:15s} {np.mean(srs):.1%} +/- {np.std(srs):.1%}     '
              f'{np.mean(rws):>7.0f}+/-{np.std(rws):<5.0f}  '
              f'{np.mean(tms)/60:.1f}m')
print('='*65)


In [ ]:
# ============================================================
# Evaluation: OOD mass sweep + statistical tests
# ============================================================
from scipy import stats as sp_stats

N_OOD_EP = 100  # episodes per mass point

# --- OOD evaluation ---
ood = {}  # method -> seed -> mass -> sr
for mn in METHODS:
    ood[mn] = {}
    for seed in SEEDS:
        if 'error' in all_results[mn].get(seed, {}):
            continue
        model = all_models[mn][seed]
        ood[mn][seed] = {}
        for mass in OOD_MASSES:
            te = PushBoxEnv(box_mass=mass)
            ok = 0
            for es in EVAL_SEEDS:
                obs, _ = te.reset(seed=es)
                done = False
                while not done:
                    act, _ = model.predict(obs, deterministic=True)
                    obs, _, term, trunc, info = te.step(act)
                    done = term or trunc
                ok += int(info.get('success', False))
            ood[mn][seed][mass] = ok / N_OOD_EP
            te.close()
        print(f'{mn} seed{seed} OOD done')

with open(f'{SAVE_DIR}/results/phase1_ood.json', 'w') as f:
    json.dump(ood, f, indent=2, default=str)
print('OOD results saved')

# --- Welch t-test: PhysRobot-SV vs PPO ---
print('\n--- Statistical tests (Welch t-test, p<0.05) ---')
for metric_label, data_src in [('In-dist SR', all_results), ('OOD mass=2.0', None)]:
    if data_src is not None:
        a = [all_results['PPO'][s]['success_rate'] for s in SEEDS
             if 'error' not in all_results['PPO'].get(s, {})]
        b = [all_results['PhysRobot-SV'][s]['success_rate'] for s in SEEDS
             if 'error' not in all_results['PhysRobot-SV'].get(s, {})]
    else:
        a = [ood['PPO'][s].get(2.0, 0) for s in SEEDS if s in ood['PPO']]
        b = [ood['PhysRobot-SV'][s].get(2.0, 0) for s in SEEDS if s in ood['PhysRobot-SV']]
    if len(a) >= 2 and len(b) >= 2:
        t, p = sp_stats.ttest_ind(a, b, equal_var=False)
        sig = '*' if p < 0.05 else 'ns'
        print(f'  {metric_label}: PPO={np.mean(a):.1%} vs SV={np.mean(b):.1%}  '
              f't={t:.2f} p={p:.3f} {sig}')
    else:
        print(f'  {metric_label}: insufficient data')

# OOD robustness score (AUC of SR-vs-mass, normalized)
print('\n--- OOD robustness score (AUC, normalized by in-dist SR) ---')
for mn in METHODS:
    seed_aucs = []
    for s in SEEDS:
        if s not in ood[mn]: continue
        srs = [ood[mn][s][m] for m in OOD_MASSES]
        auc = np.trapz(srs, OOD_MASSES)
        in_sr = ood[mn][s].get(BOX_MASS_TRAIN, 1e-9)
        seed_aucs.append(auc / max(in_sr, 1e-9))
    if seed_aucs:
        print(f'  {mn:15s}  AUC_norm = {np.mean(seed_aucs):.2f} +/- {np.std(seed_aucs):.2f}')


In [ ]:
# ============================================================
# Figures + save
# ============================================================
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams.update({'font.size': 12})

COLORS = {'PPO': '#4CAF50', 'GNS': '#2196F3', 'PhysRobot-SV': '#FF9800'}

# ---------- Fig 1: In-distribution bar chart ----------
fig, ax = plt.subplots(figsize=(7, 5))
methods_list = list(METHODS.keys())
means, stds = [], []
for mn in methods_list:
    srs = [all_results[mn][s]['success_rate'] for s in SEEDS
           if 'error' not in all_results[mn].get(s, {})]
    means.append(np.mean(srs) * 100 if srs else 0)
    stds.append(np.std(srs) * 100 if srs else 0)
bars = ax.bar(methods_list, means, yerr=stds, capsize=6,
              color=[COLORS[m] for m in methods_list], edgecolor='black')
for b, m in zip(bars, means):
    ax.text(b.get_x()+b.get_width()/2, b.get_height()+2, f'{m:.1f}%',
            ha='center', fontweight='bold')
ax.set_ylabel('Success Rate (%)')
ax.set_title('Phase 1: In-Distribution Success Rate (500K, 5 seeds)')
ax.set_ylim(0, 105)
ax.grid(axis='y', alpha=0.3)
fig.tight_layout()
fig.savefig(f'{SAVE_DIR}/figures/fig1_indist_sr.png', dpi=150)
plt.show()

# ---------- Fig 2: OOD mass sweep ----------
fig, ax = plt.subplots(figsize=(9, 6))
for mn in methods_list:
    per_mass_mean, per_mass_std = [], []
    for mass in OOD_MASSES:
        vals = [ood[mn][s][mass] for s in SEEDS if s in ood[mn]]
        per_mass_mean.append(np.mean(vals)*100 if vals else 0)
        per_mass_std.append(np.std(vals)*100 if vals else 0)
    ax.errorbar(OOD_MASSES, per_mass_mean, yerr=per_mass_std,
                fmt='o-', label=mn, color=COLORS[mn], linewidth=2,
                markersize=7, capsize=4)
ax.axvline(BOX_MASS_TRAIN, ls='--', color='gray', alpha=0.6, label='Train mass')
ax.set_xlabel('Box Mass (kg)')
ax.set_ylabel('Success Rate (%)')
ax.set_title('OOD Generalization: SR vs Box Mass (zero-shot)')
ax.legend()
ax.set_ylim(-5, 105)
ax.grid(alpha=0.3)
fig.tight_layout()
fig.savefig(f'{SAVE_DIR}/figures/fig2_ood_mass.png', dpi=150)
plt.show()

# ---------- Fig 3: Sample efficiency (first-success episode) ----------
fig, ax = plt.subplots(figsize=(7, 5))
se_data = {}
for mn in methods_list:
    eps = [all_results[mn][s].get('first_success_ep') for s in SEEDS
           if 'error' not in all_results[mn].get(s, {})]
    eps = [e for e in eps if e is not None]
    se_data[mn] = eps
box_data = [se_data[mn] for mn in methods_list]
bp = ax.boxplot(box_data, labels=methods_list, patch_artist=True)
for patch, mn in zip(bp['boxes'], methods_list):
    patch.set_facecolor(COLORS[mn])
    patch.set_alpha(0.7)
ax.set_ylabel('Episode of First Success')
ax.set_title('Sample Efficiency (lower = better)')
ax.grid(axis='y', alpha=0.3)
fig.tight_layout()
fig.savefig(f'{SAVE_DIR}/figures/fig3_sample_eff.png', dpi=150)
plt.show()

# ---------- Fig 4: Learning curves (training SR over episodes) ----------
fig, ax = plt.subplots(figsize=(9, 5))
for mn in methods_list:
    all_curves = []
    for s in SEEDS:
        rec = all_results[mn].get(s, {})
        if 'error' in rec or not rec.get('history'):
            continue
        h = rec['history']
        eps = [x[0] for x in h]
        srs = [x[1]*100 for x in h]
        all_curves.append((eps, srs))
    if all_curves:
        # align to common x
        max_len = max(len(c[0]) for c in all_curves)
        min_len = min(len(c[0]) for c in all_curves)
        if min_len > 0:
            common_eps = all_curves[0][0][:min_len]
            mat = np.array([c[1][:min_len] for c in all_curves])
            mu = mat.mean(0); sd = mat.std(0)
            ax.plot(common_eps, mu, '-', color=COLORS[mn], label=mn, lw=2)
            ax.fill_between(common_eps, mu-sd, mu+sd, color=COLORS[mn], alpha=0.15)
ax.set_xlabel('Training Episodes')
ax.set_ylabel('Cumulative Success Rate (%)')
ax.set_title('Learning Curves (shaded = 1 std)')
ax.legend()
ax.grid(alpha=0.3)
fig.tight_layout()
fig.savefig(f'{SAVE_DIR}/figures/fig4_learning_curves.png', dpi=150)
plt.show()

# ---------- summary table -> text ----------
print('\nAll figures saved to', f'{SAVE_DIR}/figures/')

summary = []
for mn in methods_list:
    srs = [all_results[mn][s]['success_rate'] for s in SEEDS
           if 'error' not in all_results[mn].get(s, {})]
    ood_2 = [ood[mn][s].get(2.0, 0) for s in SEEDS if s in ood[mn]]
    tms = [all_results[mn][s]['train_time_s'] for s in SEEDS
           if 'error' not in all_results[mn].get(s, {})]
    summary.append({
        'Method': mn,
        'SR_indist': f'{np.mean(srs):.1%} +/- {np.std(srs):.1%}' if srs else 'N/A',
        'SR_ood_2kg': f'{np.mean(ood_2):.1%} +/- {np.std(ood_2):.1%}' if ood_2 else 'N/A',
        'Time_min': f'{np.mean(tms)/60:.1f}' if tms else 'N/A',
    })
import pandas as pd
df = pd.DataFrame(summary)
print(df.to_string(index=False))
df.to_csv(f'{SAVE_DIR}/results/phase1_summary.csv', index=False)
print(f'\nDone. All artifacts in {SAVE_DIR}')
